In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/4.part'):
    shutil.rmtree('./file/4.part')

os.makedirs('./file/4.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part_code'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/3.part_code/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['Product No'] = input_['Product No'].astype(int)
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                if input_.loc[a, 'Country'] == 'US':
                    country, lang = 'us', 'en'
                elif input_.loc[a, 'Country'] == 'DE':
                    country, lang = 'de', 'de'
                elif input_.loc[a, 'Country'] == 'UK':
                    country, lang = 'gb', 'en'
                else:
                    country, lang = 'fuck', 'fuck'
    
                b = 0
                while True:
                    b += 1

                    try:
                        request_url = f'''https://www.gates.com/{country}/{lang}/search.p.{input_.loc[a, 'Product Number']}.v.{input_.loc[a, 'Part Code']}.html'''
        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                
                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))
            
                            if html.xpath('//div[@class="product-tabs__fixed"]/ul[@role="tablist"]'):
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                dict_ = json.loads(resp.text.split('new s7viewers.MixedMediaViewer(')[1].split('"localizedTexts": localisedTexts')[0].strip().removesuffix(',') + '}')

                # = = = = = = = = = = = = = = =

                list_th = [f'''Kit_{text.strip().removesuffix('#').strip().title()}''' for text in html.xpath('//div[@id="product-kit-content__content"]/div[@class="accordion-content__wrapper"]/table[@class="table c-table c-table--striped"]/thead/tr/th/text()')]

                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@id="product-kit-content__content"]/div[@class="accordion-content__wrapper"]/table[@class="table c-table c-table--striped"]/tbody/tr')

                # = = = = = = = = = = = = = = =

                dict_kit = {}
                for i, tr in enumerate(list_tr):
                    dict_kit[str(i)], list_td = {}, tr.xpath('./td')
                    for j, td in enumerate(list_td):
                        dict_kit[str(i)][list_th[j]] = td.xpath('./text()')[0].strip() if td.xpath('./text()') else ''
                        
                # = = = = = = = = = = = = = = =
        
                list_tr = html.xpath('//li[@class="nested-accordion__item nested-accordion__item--level1"]/div[@role="tabpanel"]/table[@class="table c-table c-table--split"]/tbody/tr')

                # = = = = = = = = = = = = = = =

                i, part_number, sku, title, upc, dict_information = 0, '', '', '', '', {}
                if input_.loc[a, 'Country'] == 'DE':
                    for tr in list_tr:
                        key, value = tr.xpath('./th/text()')[0].strip().removesuffix('#').strip(), tr.xpath('./td/text()')[0].strip() if tr.xpath('./td/text()') else ''
                        if key == 'Teilenummer':
                            part_number = value
                        elif key == 'Produktnummer':
                            sku = value
                        elif key == 'Beschreibung':
                            title = value
                        elif key == 'UPC':
                            upc = value
                        else:
                            dict_information[str(i)] = {key: value}
                            i += 1
                else:
                    for tr in list_tr:
                        key, value = tr.xpath('./th/text()')[0].strip().removesuffix('#').strip(), tr.xpath('./td/text()')[0].strip() if tr.xpath('./td/text()') else ''
                        if key == 'Part':
                            part_number = value
                        elif key == 'Product':
                            sku = value
                        elif key == 'Description':
                            title = value
                        elif key == 'UPC':
                            upc = value
                        else:
                            dict_information[str(i)] = {key: value}
                            i += 1
                    
                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'Product No': input_.loc[a, 'Product No'],
                                         'Product Name': input_.loc[a, 'Product Name'],
                                         'Product Number': input_.loc[a, 'Product Number'],
                                         'Country': input_.loc[a, 'Country'],
                                         'No': input_.loc[a, 'No'],
                                         'Part Number': part_number,
                                         'Type': input_.loc[a, 'Type'],
                                         'SKU': sku,
                                         'UPC': upc,
                                         'Title': title,
                                         'Url': request_url,
                                         'Json_Information': json.dumps(dict_information),
                                         'Json_Kit': json.dumps(dict_kit),
                                         'Part Code': input_.loc[a, 'Part Code'],
                                         'Asset': dict_['params']['asset'].strip()}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Product No']}.{input_.loc[a, 'Product Name']} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Code']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Product No', 'No'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/4.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Product No', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/4.part/{file}',
              f'''./file/4.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：47

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 14.Geformter Kühlmittelschlauch > 65.4275-13283
[剩余数量：42] - [当前时间：11:24:37]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 14.Geformter Kühlmittelschlauch > 55.4275-13242
[剩余数量：41] - [当前时间：11:24:38]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 14.Geformter Kühlmittelschlauch > 58.4275-13246
[剩余数量：40] - [当前时间：11:24:39]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 14.Geformter Kühlmittelschlauch > 68.4275-13300
[剩余数量：39] - [当前时间：11:24:39]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 14.Geformter Kühlmittelschlauch > 59.4275-13249
[剩余数量：38] - [当前时间：11:24:40]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 14.Geformter Kühlmittelschlauch > 69.4275-13301
[剩余数量：37] - [当前时间：11:24:40]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 14.Geformter Kühlmittelschlauch > 77.4275-13325
[剩余数量：36] - [当前时间：11:24:42]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 14.Geformter Kühlmittelschlauch > 78.4275-13326
[剩余数量：35] - [当前时间：11:24:43]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 14.Geformter Kühlmittelsc

Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 3637.73it/s]

Done ~
